# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

# 1) 10 countries with the lowest infant mortality rates

In [452]:
from xml.etree import ElementTree as ET

In [453]:
document = ET.parse('/Users/yogini/Downloads/data_wrangling_xml/data/mondial_database.xml' )

In [454]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [455]:
d1=[]
for element in document.iterfind('country'):
    #print '* ' + element.find('name').text + ':',
    if element.find('infant_mortality') is not None:
        d1.append((element.find('name').text, element.find('infant_mortality').text))

In [456]:
p1=pd.DataFrame(d1, columns=('Country', 'Rate'))

In [457]:
p1['Rate']=p1.Rate.astype(np.float)

In [622]:
p1.sort_values(['Rate']).head(10)

,Country,Rate
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


# 2) 10 cities with the largest population

In [460]:
name=""
d2=[]
root = document.getroot()
for e in root:
    if (e.tag=='country'):
        name=e.find('name').text
    for c in e:
        if (c.tag == 'population' and c.get('year')=='2011'): 
            d2.append((name,c.text))
    #print(e.tag)+"head"
        

In [461]:
p2=pd.DataFrame(d2, columns=('Country', 'Population'))

In [462]:
p2['Population']=p2.Population.astype(np.float)

In [463]:
p2.sort_values('Population',ascending=False).head(10)

,Country,Population
37,India,1.210855e+09
73,Nigeria,1.642945e+08
36,Bangladesh,1.497724e+08
10,Germany,8.021970e+07
34,Iran,7.514967e+07
7,France,6.493340e+07
33,United Kingdom,6.318218e+07
12,Italy,5.943374e+07
75,South Africa,5.177056e+07
8,Spain,4.681592e+07


In [465]:
d3=[]
for element in document.iterfind('country'):
    #print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        if ((subelement.find('population')) is not None):
            population=""
            year=0
            for subelement_p in subelement.getiterator('population'):
                    population= subelement_p.text
                    year= subelement_p.get('year')
            d3.append((subelement.find('name').text, population,year))

In [466]:
p3=pd.DataFrame(d3, columns=('City', 'Population','Year'))

In [467]:
p3['Population']=p3.Population.astype(np.float)

In [468]:
p3.sort_values(['Population'],ascending=False).head(10)

,City,Population,Year
1251,Shanghai,22315474.0,2010
707,Istanbul,13710512.0,2012
1421,Mumbai,12442373.0,2011
443,Moskva,11979529.0,2013
1250,Beijing,11716620.0,2010
2594,São Paulo,11152344.0,2010
1252,Tianjin,11090314.0,2010
974,Guangzhou,11071424.0,2010
1467,Delhi,11034555.0,2011
977,Shenzhen,10358381.0,2010


# 3) 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [472]:
d4=[]
for element in document.iterfind('country'):
    capitals_string = ''
    for subelement in element.getiterator('ethnicgroup'):
        if subelement.get('percentage') is not None:
            d4.append((element.find('name').text,subelement.text,subelement.get('percentage')))

In [473]:
p4=pd.DataFrame(d4, columns=('Country', 'EthnicGroup','Percentage'))
p4['Percentage']=p4.Percentage.astype(np.float)
p4.head()

,Country,EthnicGroup,Percentage
0,Albania,Albanian,95.0
1,Albania,Greek,3.0
2,Greece,Greek,93.0
3,Macedonia,Macedonian,64.2
4,Macedonia,Albanian,25.2


In [474]:
result = pd.merge(p4, p2,on='Country')
result['ethnic_population']=(result['Percentage']/100)*result['Population']

In [475]:
result1 = result[['ethnic_population', 'EthnicGroup']].copy()

In [476]:
result1[['ethnic_population','EthnicGroup']].groupby(['EthnicGroup'])['ethnic_population'].sum().reset_index(name='sum1').sort_values('sum1',ascending=False).head(10)

,EthnicGroup,sum1
48,Indo-Aryan,8.718156e+08
31,Dravidian,3.027137e+08
0,African,1.663920e+08
16,Bengali,1.467769e+08
42,German,7.427849e+07
33,English,5.282030e+07
62,Mediterranean Nordic,4.681592e+07
75,Persian,3.832633e+07
76,Polish,3.801842e+07
66,Mongol,3.632565e+07


# 4) name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [608]:
name=""
c_code=""
river_length=""
river_name=""
d5=[]
d6=[]
river_src_country=""
root = document.getroot()
for e in root:
    #print e.tag
    if (e.tag=='country'):
        name=e.find('name').text
        c_code= e.get('car_code')
        d5.append((name,c_code))
    if ((e.tag=='river') and (e.find('length') is not None)):
        river_name=e.find('name').text
        river_length=e.find('length').text
        river_src_country=e.find('source')
        river_src_country=river_src_country.get('country')
        river_src_country= river_src_country.split(" ")
        #print 'river:---- '+river_name
        #print "test---"+river_src_country
        d6.append(('River',river_name,river_length,river_src_country[0]))
    if ((e.tag=='lake') and (e.find('area')) is not None):
        lake_name=e.find('name').text
        lake_length=e.find('area').text
        #river_src_country=e.find('source')
        #lake_country=e.getElementsByTagName("located").item(0);
        lake_country=e.find('located')
        #print 'river:---- '+river_name
        #print "test---"+river_src_country
        #print lake_country.items()
        l_country=e.items()
        lake_cty=l_country[0][1].split(" ")
        #print l_country[0][1],lake_name
        #print lake_cty[0]
        d6.append(('lake',lake_name,lake_length,lake_cty[0]))
        #print lake_name,lake_length,lake_country
    if ((e.tag=='airport') and (e.find('elevation')) is not None):
        airport_name=e.find('name').text
        d6.append(('Airport',airport_name,e.find('elevation').text,e.get('country')))

In [609]:
d6

[('River', 'Thjorsa', '230', 'IS'),
 ('River', 'Joekulsa a Fjoellum', '206', 'IS'),
 ('River', 'Glomma', '604', 'N'),
 ('River', 'Lagen', '322', 'N'),
 ('River', 'Goetaaelv', '93', 'S'),
 ('River', 'Klaraelv', '460', 'N'),
 ('River', 'Umeaelv', '470', 'S'),
 ('River', 'Dalaelv', '520', 'S'),
 ('River', 'Vaesterdalaelv', '320', 'S'),
 ('River', 'Oesterdalaelv', '241', 'S'),
 ('River', 'Paatsjoki', '145', 'SF'),
 ('River', 'Ounasjoki', '300', 'SF'),
 ('River', 'Kemijoki', '550', 'SF'),
 ('River', 'Oulujoki', '107', 'SF'),
 ('River', 'Kymijoki', '203', 'SF'),
 ('River', 'Kokemaeenjoki', '121', 'SF'),
 ('River', 'Vuoksi', '162', 'SF'),
 ('River', 'Thames', '346', 'GB'),
 ('River', 'Maas', '925', 'F'),
 ('River', 'Loire', '1013', 'F'),
 ('River', 'Garonne', '647', 'E'),
 ('River', 'Rhone', '812', 'CH'),
 ('River', 'Saone', '480', 'F'),
 ('River', 'Doubs', '453', 'F'),
 ('River', 'Isere', '290', 'F'),
 ('River', 'Seine', '776', 'F'),
 ('River', 'Marne', '514', 'F'),
 ('River', 'Tajo', '1007'

In [610]:
p5=pd.DataFrame(d5, columns=('Country', 'Country_code'))
p6=pd.DataFrame(d6, columns=('Type','Name','Attribute', 'Country_code'))

p6['Attribute']=p6.Attribute.astype(np.float)
p5.head()

,Country,Country_code
0,Albania,AL
1,Greece,GR
2,Macedonia,MK
3,Serbia,SRB
4,Montenegro,MNE


In [611]:
merge_d = pd.merge(p5, p6,on='Country_code')
p6.head()

,Type,Name,Attribute,Country_code
0,River,Thjorsa,230.0,IS
1,River,Joekulsa a Fjoellum,206.0,IS
2,River,Glomma,604.0,N
3,River,Lagen,322.0,N
4,River,Goetaaelv,93.0,S


In [604]:
merge_d[merge_d['Name']=='Glomma']

,Country,Country_code,Type,Name,Attribute
152,Norway,N,River,Glomma,604.0


In [620]:
merge_d.groupby('Type')['Attribute'].max().reset_index(name='MAX_value')#.sort_values('MAX_value',ascending=False).head(10)

,Type,MAX_value
0,Airport,4063.0
1,River,6448.0
2,lake,386400.0


In [619]:
merge_d.groupby('Type')['Attribute'].max()

Type
Airport      4063.0
River        6448.0
lake       386400.0
Name: Attribute, dtype: float64